# PySpark Cookbook

Learn the basics of Apache Spark (PySpark) using familiar idioms from SQL


More Examples Can be found at:  https://sparkbyexamples.com/pyspark/

-------------------------------------------------------------------------------

Imports and Global Settings

In [1]:
import pyspark
from pyspark.sql import SparkSession
ENDPOINT='http://minio:9000'
AWS_ID="minio"
AWS_KEY="SU2orange!"

## Spark Session and Context 

Add S3 / Minio / Object storage to the spark session then get the spark context.
 
 - spark == SparkSession
 - cs == SparkContext

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.hadoop.fs.s3a.endpoint", ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_KEY) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext

21/09/24 13:11:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Noisy.. set the log level to output Errors only

In [3]:
sc.setLogLevel("ERROR")

Let's print some info about spark so you know which version we are using and how it was setup

In [4]:
# Print context
print('Spark Context : ', spark.sparkContext)
print('Spark Version : ', spark.sparkContext.version)
print('Spark appName :', spark.sparkContext.appName)
print('Hadoop version: ', spark.sparkContext._gateway.jvm.org.apache.hadoop.util.VersionInfo.getVersion())
print('Spark Confiuration:')
for conf in spark.sparkContext._conf.getAll():
    print(f"\t{conf[0]} = {conf[1]}")

Spark Context :  <SparkContext master=local appName=jupyter-pyspark>
Spark Version :  3.1.2
Spark appName : jupyter-pyspark
Hadoop version:  3.2.0
Spark Confiuration:
	spark.master = local
	spark.driver.port = 40199
	spark.hadoop.fs.s3a.access.key = minio
	spark.hadoop.fs.s3a.secret.key = SU2orange!
	spark.driver.host = af40586e4337
	spark.executor.id = driver
	spark.app.startTime = 1632489112086
	spark.app.id = local-1632489112547
	spark.driver.extraJavaOptions = -Dio.netty.tryReflectionSetAccessible=true
	spark.hadoop.fs.s3a.impl = org.apache.hadoop.fs.s3a.S3AFileSystem
	spark.sql.warehouse.dir = file:/home/jovyan/work/spark-warehouse/
	spark.app.name = jupyter-pyspark
	spark.hadoop.fs.s3a.path.style.access = True
	spark.rdd.compress = True
	spark.serializer.objectStreamReset = 100
	spark.submit.pyFiles = 
	spark.submit.deployMode = client
	spark.hadoop.fs.s3a.endpoint = http://minio:9000
	spark.hadoop.fs.s3a.fast.upload = True
	spark.executor.extraJavaOptions = -Dio.netty.tryReflect

## Loading Data with Spark

These examples demonstrate various ways to load data into spark dataframes. Spark dataframes will be most familiar to you as they are similar to relational tables and r dataframes.

### CSV File Format

Docs: https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrameReader.csv.html

Read in any delimited file formats

In [5]:
# traditional CSV, with header
df = spark.read.csv("s3a://ufo-sightings/*.csv",header=True, inferSchema=True).cache()
df.show(10)
df.printSchema()

Py4JJavaError: An error occurred while calling o79.csv.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2595)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3269)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:377)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.sc

In [70]:
# A Pipe separated file, no header, So we rename some columns
df = spark.read.csv("s3a://tweets/tweets.psv",header=False, inferSchema=True,sep='|')
df = df.withColumnRenamed('_c4','tweet_text') \
    .withColumnRenamed("_c3","twitter_user")
df.show(10)

+-------------------+--------------------+--------------------+------------+--------------------+
|                _c0|                 _c1|                 _c2|twitter_user|          tweet_text|
+-------------------+--------------------+--------------------+------------+--------------------+
|2845428583999282239|1.4337661612984276E9|Mon Jun 08 08:22:...|    rovlight|Why so horrible d...|
|1658183905022391067|1.4298210344679017E9|Thu Apr 23 16:30:...|       sladd|Just placed an or...|
| 973476786498736360|1.4421079524352274E9|Sat Sep 12 21:32:...|     rdeboat|Worst purchase ev...|
| 402981967046309846| 1.447381758722716E9|Thu Nov 12 21:29:...|      sofewe|I love you, @fudg...|
|1397754748167468823|1.4355730173666306E9|Mon Jun 29 06:16:...|      balott|Glad I bought my ...|
|4317414692342006509| 1.441177812164332E9|Wed Sep 02 03:10:...|    ojouglad|The Electronics d...|
|1397715809532042193|1.4361083498747828E9|Sun Jul 05 10:59:...|    edetyers|Who uses the fudg...|
| 268216688744405461

Don't like `show()` you can always convert the output to a Pandas Dataframe.

In [117]:
df.show(10)
df.toPandas().sample(10) # get 10 random samples

+--------------------+--------------------+-------------------+--------------------+----------+
|          created_at| created_at_unixtime|                 id|                text|      user|
+--------------------+--------------------+-------------------+--------------------+----------+
|Sun Nov 22 14:44:...|1.4482214566646008E9|3731785240073317438|Glad I bought my ...|    jpoole|
|Mon Aug 24 05:05:...| 1.440407147033846E9|3600730356622213650|Techical support ...|    jpoole|
|Sat Nov 28 00:19:...|1.4486879496857476E9|4211873369158903141|Do you enjoy digi...|     tpani|
|Thu Jan 29 23:27:...|1.4225920780295255E9| 803003777759256860|The Electronics d...| lhvmeehom|
|Sat Feb 07 16:29:...|1.4233445491064806E9|2073670596081008741|Hey fudgemart, wh...|  mmelator|
|Wed May 20 11:27:...|1.4321356508858037E9|4179972589054475991|I like the Fudgem...|  mmelator|
|Sat Sep 26 05:10:...| 1.443258613345817E9|3924522994216591792|@fudgemart is bet...|   rdeboat|
|Mon May 25 12:21:...|   1.4325709074274

,created_at,created_at_unixtime,id,text,user
60,Tue May 05 13:58:24 +0000 2015,1.430849e+09,1682524703217110597,Came home to a big smile my #fudgemart package...,edetyers
131,Fri Dec 18 20:48:13 +0000 2015,1.450490e+09,3128030551379405713,Fudgemart just is.,edetyers
112,Wed Jul 01 12:38:55 +0000 2015,1.435769e+09,1029113340942294688,Just placed an order today. thx @fudgemart,bmelator
136,Thu Jun 18 12:54:44 +0000 2015,1.434646e+09,2976189753055105294,Shopping on #fudgemart website is carefree.,sbellum
100,Mon Sep 28 09:20:38 +0000 2015,1.443446e+09,4366340990425716647,@fudgemart is better than @amazeen,edetyers
154,Thu Feb 26 00:53:55 +0000 2015,1.424930e+09,2873970204275959017,"Wonderful books selection, @fudgemart",edetyers
189,Wed Jul 08 14:36:48 +0000 2015,1.436381e+09,1085812989444393574,"My shopping experience was delightful, @fudgemart",rdeboat
20,Mon Nov 23 17:12:23 +0000 2015,1.448317e+09,3067158767857887238,Nothing like the feeling of helplessness from ...,sofewe
1,Mon Aug 24 05:05:47 +0000 2015,1.440407e+09,3600730356622213650,"Techical support for my new computer as A+, th...",jpoole
93,Sat Feb 07 08:10:25 +0000 2015,1.423315e+09,1942116576821067942,The books department at #fudgemart is amazing....,edetyers


### JSON File Format

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrameReader.json.html


In [118]:
# Simple JSON With the JSON format, printSchema() is a friend indeed!
df = spark.read.json("s3a://tweets/tweets.json")
df.show(10)
df.printSchema()


+--------------------+--------------------+-------------------+--------------------+----------+
|          created_at| created_at_unixtime|                 id|                text|      user|
+--------------------+--------------------+-------------------+--------------------+----------+
|Sun Nov 22 14:44:...|1.4482214566646008E9|3731785240073317438|Glad I bought my ...|    jpoole|
|Mon Aug 24 05:05:...| 1.440407147033846E9|3600730356622213650|Techical support ...|    jpoole|
|Sat Nov 28 00:19:...|1.4486879496857476E9|4211873369158903141|Do you enjoy digi...|     tpani|
|Thu Jan 29 23:27:...|1.4225920780295255E9| 803003777759256860|The Electronics d...| lhvmeehom|
|Sat Feb 07 16:29:...|1.4233445491064806E9|2073670596081008741|Hey fudgemart, wh...|  mmelator|
|Wed May 20 11:27:...|1.4321356508858037E9|4179972589054475991|I like the Fudgem...|  mmelator|
|Sat Sep 26 05:10:...| 1.443258613345817E9|3924522994216591792|@fudgemart is bet...|   rdeboat|
|Mon May 25 12:21:...|   1.4325709074274

In [86]:
# Formatted (a.k.a. Pretty Printed) JSON might be multi-line,
df = spark.read.json("s3a://json-samples/fudgemart-products.json", multiLine=True)
df.show(10)

+----------------+----------+--------------------+--------------------+-----------+----------------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|  vendor_website|
+----------------+----------+--------------------+--------------------+-----------+----------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|            null|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|            null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|            null|
|        Clothing|         4|         Dri-Fit Tee|                20.0|      Mikey|http://mikee.com|
|        Clothing|         5|       Running Pants|                35.0|      Mikey|http://mikee.com|
|        Clothing|         6|          Wool Socks|                 8.0|      Mikey|http://mikee.com|
|        Clothing|         7|      Squeaky Sneaks|                65.0|      Mikey|http://m

In [119]:
# Nested JSON is handed really well... we will learn how to query this later!!
df = spark.read.json("s3a://json-samples/google-places.json", multiLine=True)
df.printSchema()

root
 |-- business_status: string (nullable = true)
 |-- geometry: struct (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lng: double (nullable = true)
 |    |-- viewport: struct (nullable = true)
 |    |    |-- northeast: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lng: double (nullable = true)
 |    |    |-- southwest: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lng: double (nullable = true)
 |-- icon: string (nullable = true)
 |-- icon_background_color: string (nullable = true)
 |-- icon_mask_base_uri: string (nullable = true)
 |-- name: string (nullable = true)
 |-- opening_hours: struct (nullable = true)
 |    |-- open_now: boolean (nullable = true)
 |-- photos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- height: long (nullable = true)
 |    |    |-- html_attributi

## Spark Dataframe Transformations

This section will cover commone DataFrame transofrmations

In [166]:
prods = spark.read.json("s3a://json-samples/fudgemart-products.json", multiLine=True)
prods.show(10)
prods.printSchema()

+----------------+----------+--------------------+--------------------+-----------+----------------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|  vendor_website|
+----------------+----------+--------------------+--------------------+-----------+----------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|            null|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|            null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|            null|
|        Clothing|         4|         Dri-Fit Tee|                20.0|      Mikey|http://mikee.com|
|        Clothing|         5|       Running Pants|                35.0|      Mikey|http://mikee.com|
|        Clothing|         6|          Wool Socks|                 8.0|      Mikey|http://mikee.com|
|        Clothing|         7|      Squeaky Sneaks|                65.0|      Mikey|http://m

### Column Projections

Use the `.select()` function for column projections. Here are three differen ways to project a column.

In [167]:
prods.select("product_name",prods.product_name, prods["product_name"]).show(10)

+--------------------+--------------------+--------------------+
|        product_name|        product_name|        product_name|
+--------------------+--------------------+--------------------+
|Straight Claw Hammer|Straight Claw Hammer|Straight Claw Hammer|
|       Sledge Hammer|       Sledge Hammer|       Sledge Hammer|
|     Rip Claw Hammer|     Rip Claw Hammer|     Rip Claw Hammer|
|         Dri-Fit Tee|         Dri-Fit Tee|         Dri-Fit Tee|
|       Running Pants|       Running Pants|       Running Pants|
|          Wool Socks|          Wool Socks|          Wool Socks|
|      Squeaky Sneaks|      Squeaky Sneaks|      Squeaky Sneaks|
|          Cool Jeans|          Cool Jeans|          Cool Jeans|
|        Denim Jacket|        Denim Jacket|        Denim Jacket|
|      Leather Jacket|      Leather Jacket|      Leather Jacket|
+--------------------+--------------------+--------------------+
only showing top 10 rows



### Derived columns 

Use `.withColumn()` to derive a new column

In [170]:
prods.withColumn("new_column", prods['product_id'] + 10000).show(10)

+----------------+----------+--------------------+--------------------+-----------+----------------+----------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|  vendor_website|new_column|
+----------------+----------+--------------------+--------------------+-----------+----------------+----------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|            null|     10001|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|            null|     10002|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|            null|     10003|
|        Clothing|         4|         Dri-Fit Tee|                20.0|      Mikey|http://mikee.com|     10004|
|        Clothing|         5|       Running Pants|                35.0|      Mikey|http://mikee.com|     10005|
|        Clothing|         6|          Wool Socks|                 8.0|      Mikey|http://mikee.com|    

This can be done with a user-defined function, too.

- first create your python function
- then register it as a Spark UDF (User-Defined Function) with appropriate Spark return type
- invoke the function in the `withColumn()` call

In [171]:
from pyspark.sql import functions as func
from pyspark.sql.types import *

def price_category(price):
    if price < 20:
        return "cheap"
    elif price > 100:
        return "expensive"
    else:
        return "moderate"

price_category_func = func.udf(price_category, returnType=StringType() )

prods.select('product_name','product_retail_price').withColumn('price_category', price_category_func(prods.product_retail_price)).show(10)
    

+--------------------+--------------------+--------------+
|        product_name|product_retail_price|price_category|
+--------------------+--------------------+--------------+
|Straight Claw Hammer|               15.95|         cheap|
|       Sledge Hammer|               21.95|      moderate|
|     Rip Claw Hammer|               19.95|         cheap|
|         Dri-Fit Tee|                20.0|      moderate|
|       Running Pants|                35.0|      moderate|
|          Wool Socks|                 8.0|         cheap|
|      Squeaky Sneaks|                65.0|      moderate|
|          Cool Jeans|                45.0|      moderate|
|        Denim Jacket|                60.0|      moderate|
|      Leather Jacket|                95.0|      moderate|
+--------------------+--------------------+--------------+
only showing top 10 rows



### Projecting nested JSON Columns

projecting nested JSON is simple use the dot operator `.` to navigate the heirarchy

In [172]:
places = spark.read.json("s3a://json-samples/google-places.json", multiLine=True)
places.printSchema()

root
 |-- business_status: string (nullable = true)
 |-- geometry: struct (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lng: double (nullable = true)
 |    |-- viewport: struct (nullable = true)
 |    |    |-- northeast: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lng: double (nullable = true)
 |    |    |-- southwest: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- lng: double (nullable = true)
 |-- icon: string (nullable = true)
 |-- icon_background_color: string (nullable = true)
 |-- icon_mask_base_uri: string (nullable = true)
 |-- name: string (nullable = true)
 |-- opening_hours: struct (nullable = true)
 |    |-- open_now: boolean (nullable = true)
 |-- photos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- height: long (nullable = true)
 |    |    |-- html_attributi

In [173]:
# get some columns!
places.select('name','geometry.location.lat',places.geometry.location.lng, places['types']).show()

+--------------------+-----------------+---------------------+--------------------+
|                name|              lat|geometry.location.lng|               types|
+--------------------+-----------------+---------------------+--------------------+
|            Syracuse|       43.0481221|   -76.14742439999999|[locality, politi...|
|Crowne Plaza Syra...|       43.0476078|          -76.1417642|[lodging, point_o...|
|  The Parkview Hotel|       43.0476157|           -76.140986|[lodging, point_o...|
|Jefferson Clinton...|       43.0472894|   -76.15385049999999|[lodging, point_o...|
|Courtyard by Marr...|       43.0488846|          -76.1561175|[lodging, point_o...|
|Quality Inn & Sui...|43.05264399999999|   -76.14681999999999|[lodging, point_o...|
| Syracuse University|       43.0391534|          -76.1351158|[university, poin...|
|Collegian Hotel &...|       43.0464172|   -76.13539879999999|[lodging, restaur...|
|  Dinosaur Bar-B-Que|       43.0526411|   -76.15469379999999|[restaurant, f

### Projecting nested JSON Columns of Arrays

we need to use `explode()` to deal with nested lists of multiple values

In [1]:
from pyspark.sql.functions import explode,split
places.select('name',explode("photos")).printSchema()
places.select('name',explode("photos")).select("name","col.photo_reference").show()

NameError: name 'places' is not defined

### Row Selections

Filter rows using `.filter()`


In [177]:
prods.filter("product_category='Hardware'").show(10)

+----------------+----------+--------------------+--------------------+------------------+--------------+
|product_category|product_id|        product_name|product_retail_price|       vendor_name|vendor_website|
+----------------+----------+--------------------+--------------------+------------------+--------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|           Stanlee|          null|
|        Hardware|         2|       Sledge Hammer|               21.95|           Stanlee|          null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|           Stanlee|          null|
|        Hardware|        29|18v Drill Driver Set|                90.0|           Stanlee|          null|
|        Hardware|        30|19.2v Drill Drive...|                90.0|Blackened-Deckhand|          null|
|        Hardware|        31|       10" Miter Saw|               200.0|Blackened-Deckhand|          null|
|        Hardware|        32|         Lazer Le

In [178]:
prods.filter(prods.product_category == 'Hardware').show(10)

+----------------+----------+--------------------+--------------------+------------------+--------------+
|product_category|product_id|        product_name|product_retail_price|       vendor_name|vendor_website|
+----------------+----------+--------------------+--------------------+------------------+--------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|           Stanlee|          null|
|        Hardware|         2|       Sledge Hammer|               21.95|           Stanlee|          null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|           Stanlee|          null|
|        Hardware|        29|18v Drill Driver Set|                90.0|           Stanlee|          null|
|        Hardware|        30|19.2v Drill Drive...|                90.0|Blackened-Deckhand|          null|
|        Hardware|        31|       10" Miter Saw|               200.0|Blackened-Deckhand|          null|
|        Hardware|        32|         Lazer Le

In [179]:
cat = 'Hardware'
vend = 'Stanlee'
prods.filter(f"product_category='{cat}' and vendor_name='{vend}'").show(10)

+----------------+----------+--------------------+--------------------+-----------+--------------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|vendor_website|
+----------------+----------+--------------------+--------------------+-----------+--------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|          null|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|          null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|          null|
|        Hardware|        29|18v Drill Driver Set|                90.0|    Stanlee|          null|
|        Hardware|        35|     Cold Chisel Set|                10.0|    Stanlee|          null|
|        Hardware|        36|     Screwdriver Set|                10.0|    Stanlee|          null|
|        Hardware|        37|       Drill Bit Set|                25.0|    Stanlee|          null|
+---------

In [182]:
cat = 'Hardware'
vend = 'Stanlee'
prods.filter((prods.product_category==cat) & (prods.vendor_name ==vend)).show(10)

+----------------+----------+--------------------+--------------------+-----------+--------------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|vendor_website|
+----------------+----------+--------------------+--------------------+-----------+--------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|          null|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|          null|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|          null|
|        Hardware|        29|18v Drill Driver Set|                90.0|    Stanlee|          null|
|        Hardware|        35|     Cold Chisel Set|                10.0|    Stanlee|          null|
|        Hardware|        36|     Screwdriver Set|                10.0|    Stanlee|          null|
|        Hardware|        37|       Drill Bit Set|                25.0|    Stanlee|          null|
+---------

### Grouping and Aggregate functions

When you use a  `groupBy()` function, you must include an aggregate like`sum(), min(), max(), count(), avg(), or agg()`


In [184]:
prods.groupBy("product_category").avg("product_retail_price").show(10)

+----------------+-------------------------+
|product_category|avg(product_retail_price)|
+----------------+-------------------------+
|  Sporting Goods|                     58.3|
|      Housewares|                     25.0|
|     Electronics|        458.8181818181818|
|        Clothing|       36.785714285714285|
|        Hardware|       104.44999999999999|
+----------------+-------------------------+



In [205]:
from pyspark.sql.functions import sum,avg,max,min,mean,count

prods.groupBy(prods.vendor_name, "product_category") \
    .agg(  avg("product_retail_price").alias("avg_price"), count("*").alias("count")   ).show(10)

+------------------+----------------+------------------+-----+
|       vendor_name|product_category|         avg_price|count|
+------------------+----------------+------------------+-----+
|           Stanlee|        Clothing|              23.0|    2|
|         Leaveeyes|        Clothing|              56.0|    4|
|             Mikey|        Clothing|              32.0|    4|
|           Weebock|  Sporting Goods|              95.0|    3|
|           Stanlee|        Hardware|             27.55|    7|
|           Weebock|        Clothing|             29.25|    4|
|             Soney|     Electronics|             757.5|    6|
|Blackened-Deckhand|        Hardware|194.16666666666666|    6|
|Blackened-Deckhand|      Housewares|              25.0|    5|
|        Mikerosoft|     Electronics|             100.4|    5|
+------------------+----------------+------------------+-----+
only showing top 10 rows



### Combining dataframes 

You can use the  `union()` function to merge dataframes by row. Both data frames must have the same number of columns.


In [209]:
prod1 = prods.filter("vendor_name='Mikerosoft'")
prod2 = prods.filter("vendor_name='Mikey'")
prod1.union(prod2).show(10)

+----------------+----------+------------------+--------------------+-----------+--------------------+
|product_category|product_id|      product_name|product_retail_price|vendor_name|      vendor_website|
+----------------+----------+------------------+--------------------+-----------+--------------------+
|     Electronics|        24|         PC Webcam|                20.0| Mikerosoft|http://www.mikero...|
|     Electronics|        25|    Computer Mouse|                10.0| Mikerosoft|http://www.mikero...|
|     Electronics|        26|Ergonomic Keyboard|                22.0| Mikerosoft|http://www.mikero...|
|     Electronics|        27|   20" LCD Monitor|               300.0| Mikerosoft|http://www.mikero...|
|     Electronics|        28|   17" LCD Monitor|               150.0| Mikerosoft|http://www.mikero...|
|        Clothing|         4|       Dri-Fit Tee|                20.0|      Mikey|    http://mikee.com|
|        Clothing|         5|     Running Pants|                35.0|    

### Join dataframes on matching columns

You can use the  `join()` function to match rows on a common column, similar to an SQL join.

arguments:

- `other`: Right side of the join
- `on`: a string for the join column name
- `how`: default inner. Must be one of `inner, cross, outer,full, full_outer, left, left_outer, right, right_outer,left_semi, left_anti`

In [215]:
avg_prices = prods.groupBy("product_category").avg("product_retail_price")
joined = prods.join(avg_prices, avg_prices.product_category == prods.product_category, how="inner")
joined.show(10)


+----------------+----------+--------------------+--------------------+-----------+----------------+----------------+-------------------------+
|product_category|product_id|        product_name|product_retail_price|vendor_name|  vendor_website|product_category|avg(product_retail_price)|
+----------------+----------+--------------------+--------------------+-----------+----------------+----------------+-------------------------+
|        Hardware|         1|Straight Claw Hammer|               15.95|    Stanlee|            null|        Hardware|       104.44999999999999|
|        Hardware|         2|       Sledge Hammer|               21.95|    Stanlee|            null|        Hardware|       104.44999999999999|
|        Hardware|         3|     Rip Claw Hammer|               19.95|    Stanlee|            null|        Hardware|       104.44999999999999|
|        Clothing|         4|         Dri-Fit Tee|                20.0|      Mikey|http://mikee.com|        Clothing|       36.785714285

### Sorting dataframes 

You can use the  `sort()` function to sort by columns.

In [219]:
prods.sort("product_id", ascending =False).show(10)

+----------------+----------+------------------+--------------------+------------------+--------------------+
|product_category|product_id|      product_name|product_retail_price|       vendor_name|      vendor_website|
+----------------+----------+------------------+--------------------+------------------+--------------------+
|      Housewares|        53|           Blender|                45.0|Blackened-Deckhand|                null|
|      Housewares|        52|        Steam Iron|                15.0|Blackened-Deckhand|                null|
|  Sporting Goods|        51|       Soccer Ball|                45.0|             Mikey|    http://mikee.com|
|  Sporting Goods|        50|       Sport Cycle|               255.0|           Weebock|http://www.weeboc...|
|  Sporting Goods|        49|         Pedometer|                10.0|           Weebock|http://www.weeboc...|
|  Sporting Goods|        48|     Heart Monitor|                20.0|           Weebock|http://www.weeboc...|
|  Sportin

## Spark SQL 

Spark allows you to complete transformations with SQL by creating table objects. The lifetime of these table objects are tied to the spark session.

In [220]:
ufo = spark.read.csv("s3a://ufo-sightings/*.csv",header=True, inferSchema=True)
ufo.createOrReplaceTempView("ufo_table")
spark.sql("select City, State, Summary from ufo_table where State='PA'").show()

+--------------------+-----+--------------------+
|                City|State|             Summary|
+--------------------+-----+--------------------+
|Salunga/Landisvil...|   PA|Pennsylvania tria...|
|                York|   PA|Myself and 2 frie...|
|        Tannersville|   PA|Bright ball of li...|
|        Tannersville|   PA|Blue/white orb sp...|
|              DuBois|   PA|Increase from sof...|
|          Washington|   PA|I saw a bright li...|
|           Dillsburg|   PA|3 motionless, bri...|
|  Lower Mount Bethel|   PA|Triangular craft ...|
|             Hanover|   PA|6 red lights movi...|
|         Churchville|   PA|After hearing hel...|
|        Philadelphia|   PA|4 orange lights s...|
|         Tunkhannock|   PA|On a ride home af...|
|        Philadelphia|   PA|((HOAX??)) One or...|
|            Starview|   PA|Bright flash, fol...|
|         Morrisville|   PA|Red Lights triang...|
|            Fredonia|   PA|3 lights, non-bli...|
|           Allentown|   PA|2 twin spheres of...|


You can see all the tables in your spark session:

In [126]:
spark.catalog.listTables()

[Table(name='ufo_table', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='ufos', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [221]:
# Lots of ways to turn the table back into a dataframe
ufo2 = spark.read.table("ufo_table")
ufo2.show(10)

# Obviously slower...
spark.sql("select * from ufo_table").show(10)

+-------------+--------------------+-----+--------+-------------+--------------------+--------+
|  Date / Time|                City|State|   Shape|     Duration|             Summary|  Posted|
+-------------+--------------------+-----+--------+-------------+--------------------+--------+
|2/29/16 23:45|        Harbor Beach|   MI|   Light|     1 minute|Yellow/white ball...|3/4/2016|
|2/29/16 23:30|           Sebastian|   FL|Triangle|20-40 minutes|6 low flying craf...|3/4/2016|
|2/29/16 23:00|Salunga/Landisvil...|   PA|Triangle| 5-15 minutes|Pennsylvania tria...|3/4/2016|
|2/29/16 22:00|                York|   PA|Triangle|   30 minutes|Myself and 2 frie...|3/4/2016|
|2/29/16 21:35|              Joliet|   IL| Unknown|   10 minutes|At approximately ...|3/4/2016|
|2/29/16 21:00|London (Buckingha...| null|Triangle|   45 minutes|Black triangle, n...|4/1/2016|
|2/29/16 20:45|  Port Hope (Canada)|   ON|   Light|   20 minutes|Strange glowing l...|3/4/2016|
|2/29/16 20:29|       Warner Robins|   G

REALLY COMPLEX SQL IS SUPPORTED!!!

In [251]:
prods.createOrReplaceTempView("products_table")
avg_prices = prods.groupBy("product_category").avg("product_retail_price").withColumnRenamed("avg(product_retail_price)","avg_retail_price")
avg_prices.createOrReplaceTempView("products_avg_prices_table")
sql_text = '''
select p.product_category, p.product_name, p.product_retail_price, ap.avg_retail_price, product_retail_price-avg_retail_price as difference_in_price
    from products_table p
        join products_avg_prices_table ap 
            on p.product_category = ap.product_category
    where p.product_category in ('Hardware','Clothing')
'''
df_result = spark.sql(sql_text)
df_result.show(10)

+----------------+--------------------+--------------------+------------------+-------------------+
|product_category|        product_name|product_retail_price|  avg_retail_price|difference_in_price|
+----------------+--------------------+--------------------+------------------+-------------------+
|        Hardware|Straight Claw Hammer|               15.95|104.44999999999999| -88.49999999999999|
|        Hardware|       Sledge Hammer|               21.95|104.44999999999999| -82.49999999999999|
|        Hardware|     Rip Claw Hammer|               19.95|104.44999999999999| -84.49999999999999|
|        Clothing|         Dri-Fit Tee|                20.0|36.785714285714285|-16.785714285714285|
|        Clothing|       Running Pants|                35.0|36.785714285714285|-1.7857142857142847|
|        Clothing|          Wool Socks|                 8.0|36.785714285714285|-28.785714285714285|
|        Clothing|      Squeaky Sneaks|                65.0|36.785714285714285| 28.214285714285715|


## Writing Data with Spark

With yoru transforms complete you can write data an DataFrame in a variety of formats:
    
- `df.write.csv` = csv / delimitied
- `df.write.json` = JSON
- `df.write.orc` = ORC (Optimized Row-Column) File Format with schema
- `df.write.parquet` = Parquet File format - efficient and stores schema with data.


Files are written to a folder... the "Hadoop Way"


In [256]:
#write locally - check your work folder
df_result.write.csv('example.csv', mode = 'overwrite')
df_result.write.parquet("example.parquet", mode = 'overwrite')

In [259]:
# Write to S3 Like Block Storage "demo-bucket"
df_result.write.parquet("s3a://demo-bucket/example.parquet",mode='overwrite')

In [261]:
spark.read.parquet("s3a://demo-bucket/example.parquet").toPandas()

,product_category,product_name,product_retail_price,avg_retail_price,difference_in_price
0,Hardware,Straight Claw Hammer,15.95,104.450000,-88.500000
1,Hardware,Sledge Hammer,21.95,104.450000,-82.500000
2,Hardware,Rip Claw Hammer,19.95,104.450000,-84.500000
3,Clothing,Dri-Fit Tee,20.00,36.785714,-16.785714
4,Clothing,Running Pants,35.00,36.785714,-1.785714
5,Clothing,Wool Socks,8.00,36.785714,-28.785714
6,Clothing,Squeaky Sneaks,65.00,36.785714,28.214286
7,Clothing,Cool Jeans,45.00,36.785714,8.214286
8,Clothing,Denim Jacket,60.00,36.785714,23.214286
9,Clothing,Leather Jacket,95.00,36.785714,58.214286
